# Laboratorium 6 - Singular Value Decomposition

## Anna Nosek
nr indeksu 305381

In [1]:
import wikipediaapi as wa
import os.path
import string
import scipy.sparse as sparse
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.preprocessing import normalize
from sklearn.decomposition import TruncatedSVD
from collections import defaultdict
import json

# nltk.download('punkt')
# nltk.download('stopwords')

wiki_wiki = wa.Wikipedia('en')
max_depth = 2
res_dir = "res"
dir = res_dir + os.sep + "stoner"
ps = PorterStemmer()


def valid_title(title):
    words = {"Wikipedia", "Help", "Category", ":", "\"", "/", "?", "List", "*", "<", ">", "|"}
    for word in words:
        if word in title:
            return False
    return True


def visit_links(page, rec_level):
    if rec_level > max_depth:
        return

    links = page.links
    for key in sorted(links.keys()):
        title = links[key].title
        if valid_title(title):
            visit_page(title, rec_level + 1)
            

def print_categorymembers(categorymembers, level=0, max_level=1):
    for c in categorymembers.values():
        print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
        if c.ns == wa.Namespace.CATEGORY and level < max_level:
            print_categorymembers(c.categorymembers, level=level + 1, max_level=max_level)


def visit_page(page, rec_level):
    
    page_py = wiki_wiki.page(page)
    file_name = dir + page_py.title + ".txt"
    
    if os.path.exists(file_name):
        return
    
    words = page_py.text.split()[:200]
    if len(words) > 100:
        
        with open(dir + os.sep + page_py.title + ".txt", 'w+', encoding="utf-8") as file:
            file.write(page_py.title+"\n")
            for word in words:
                file.write(word+' ')
        visit_links(page_py, rec_level)


def get_files():
    categories = wiki_wiki.page("Category:Stoner rock musical groups")

    for member in categories.categorymembers:
        visit_page(member, 0)

## Dane

Aby pozyskać dokumenty wykorzystano Wikipedia API. Za jego pomocą pobrano około 50 tysięcy artykułów z Wikipedii (zapisano pierwsze 200 słów każdego z artykułów wraz z ich tytułem). Artykuły pobierano w następujący sposób: rozpoczęto od listy zespołów z kategorii Stoner rock musical groups, a następnie rekurencyjnie w każdym arykule wchodzono do kolejnych odnośników do artykułów Wikipedii. Ominięto artykuły mające mało treści, a także takie, które były kolejnymi listami. Uzyskane w ten sposób dokumenty nie są spójne tematycznie. Z maksymalnym poziomem zagłębienia rekurencji równym 2 uzyskano już różnorodną tematykę otrzymanych tekstów.

Link do wykorzystanego API:
https://github.com/martin-majlis/Wikipedia-API

In [2]:
def get_terms(dir):
    terms = set({})
    document_count = 0
    documents_dict = {}
    i = 0
    for subdir, dirs, files in os.walk(dir):
        for file in files:
            documents_dict[file] = i
            i += 1
            file_path = dir + os.sep + file
            document_count+=1
            with open(file_path, encoding="utf-8") as f:
                text = f.read()
                words = text_to_words(text)
                for word in words:
                    terms.add(word)
    return np.sort(list(terms)), documents_dict

def is_ascii(s):
    return all(ord(c) < 128 for c in s)

def text_to_words(text):
    text = clear_text(text)
    words = word_tokenize(text)
    words = remove_stop_words(words)
    return words


def clear_text(text):
    text = "".join([c for c in text if c not in string.punctuation])
    return text.lower()


def remove_stop_words(words):
    stop_words = set(stopwords.words('english'))
    words = [ps.stem(w) for w in words if (w not in stop_words and is_ascii(w))]
    return words

Powyższy kod odpowiada za wstępne przetworzenie dokumentów i uzyskanie słownika termów. Słownik ten został uzyskany jako unia wszystkich słów występujących we wszystkich dokumentach. Usunięto znaki interpunkcyjne, wszystkie litery zamieniono na małe i zastosowano stemming (PorterStemmer z bilioteki nltk). Usunięto takze stop words (listę stop words uzyskano z biblioteki nltk). W funkcji get_terms stworzono także słownik dokumentów, aby w późniejszych etapach mieć łatwy dostęp do indeksu danego dokumentu.

In [3]:
terms, documents_dict = get_terms(dir)
print(terms)

['0' '00' '000' ... 'zzap64' 'zzebra' 'zzzzz']


In [4]:
print(len(terms))
terms_dict = {}
for i in range(len(terms)):
    terms_dict[terms[i]] = i

197360


Za pomocą słownika zaindeksowano posortowane termy. Następnie oba słowniki zapisano do plików o rozszerzeniu .json.

In [5]:
print(len(terms_dict))
print(len(documents_dict))

197360
50293


In [6]:
json.dump(terms_dict, open( res_dir + os.sep + "terms_dict.json", 'w' ))
json.dump(documents_dict, open( res_dir + os.sep + "documents_dict.json", 'w' ))

In [7]:
def load_dicts():
    terms_dict = json.load( open( res_dir + os.sep + "terms_dict.json" ) )
    documents_dict = json.load( open( res_dir + os.sep + "documents_dict.json" ) )
    return terms_dict, documents_dict

## Obliczenie bag of words i term by document matrix

In [8]:
def get_term_by_document_matrix(terms_dict, dir, documents_dict):
    
    terms_count = len(terms_dict)
    documents_count = len(documents_dict)
    
    tbd = sparse.lil_matrix((terms_count, documents_count))
    for subdir, dirs, files in os.walk(dir):
        for file in files:
            file_path = dir + os.sep + file
            with open(file_path, encoding="utf-8") as f:
                for word in text_to_words(f.read()):
                    tbd[terms_dict[word], documents_dict[file]] += 1
    return sparse.csc_matrix(tbd)

In [9]:
tbd = get_term_by_document_matrix(terms_dict, dir, documents_dict)
sparse.save_npz(res_dir + os.sep + 'tbd_no_idf.npz', tbd)

In [10]:
print(tbd.shape)

(197360, 50293)


Utworzono macierz term by document poprzez wyznaczenie dla każdego dokumentu wektora bag of words. Do utworzenia macierzy term by document wykorzystano macierz lil z scipy.sparse.

In [11]:
def apply_idf(tbd, terms_count, documents_count):
    idf = []
    tbd_csr = sparse.csr_matrix(tbd)
    i = 0
    for row in tbd_csr:
        doc_count = row.count_nonzero()
        idf_i = np.log(documents_count/doc_count)
        idf.append(idf_i)
        row = row * idf_i
    return idf, tbd_csr

In [12]:
idf, tbd_csr = apply_idf(tbd, len(terms_dict), len(documents_dict))

In [13]:
tbd_csc = sparse.csc_matrix(tbd_csr)
sparse.save_npz(res_dir + os.sep + 'tbd.npz', tbd_csc)

Następnie wyznaczono inverse document frequency dla każdego dokumentu. Liczbę dokumentów, w której wystąpiło dane słowo obliczno jako liczbę niezerowych komórek w rzędzie macierzy term by document. Tak przetworzoną macierz term by document zapisano do pliku.

In [14]:
def get_col_norms(tbd, documents_count):
    
    col_norms = []
    
    for i in range(documents_count):
        col_norm = sparse.linalg.norm(tbd.getcol(i))
        col_norms.append(col_norm)
    return sparse.csc_matrix(col_norms)

In [15]:
col_norms = get_col_norms(tbd_csc, len(documents_dict))

In [16]:
sparse.save_npz(res_dir + os.sep + 'col_norms.npz', col_norms)

In [17]:
def print_results(results):
    for res in results:
        file_path = dir+os.sep+res[1]
        with open(file_path, encoding="utf-8") as f:
            text = f.read()
            print(text)
            print("\n")

In [18]:
def query_to_bow(query, terms_dict):

    query_words = text_to_words(query)
    query_words = clean_query(query_words, terms_dict)

    if not query_words:
        return []

    query_bow = sparse.lil_matrix((len(terms_dict), 1))

    for word in query_words:
        query_bow[terms_dict[word], 0] += 1
    return query_bow

def clean_query(query_words, terms_dict):
    query_words = [word for word in query_words if word in terms_dict]
    return query_words

Powyższe funkcje, używane we wszystkich wariantach wyszukiwarki przekształca zapytanie w wektor bag of words. Najpierw poddaje tekst tym samym przekształceniom, co dokumenty (usunięcie interpunkcji i stop words, stemming), usuwa z zapytania słowa niebędące w słowniku, a następnie tworzy wektor bag of words w postaci macierzy rzadkiej.

## Wyszukiwanie bez wcześniejszej normalizacji macierzy

In [19]:
def find_documents_not_normalized(query, k):
    
    terms_dict, documents_dict = load_dicts()
    
    query_bow = query_to_bow(query, terms_dict)
    
    if query_bow == []:
        return []
    
    query_bow_t = query_bow.transpose()
    
    tbd = sparse.load_npz(res_dir + os.sep + 'tbd.npz')
    col_norms = sparse.load_npz(res_dir + os.sep + 'col_norms.npz')
    results = []
    
    query_norm = sparse.linalg.norm(query_bow)
    
    for document in documents_dict.keys():
        i = documents_dict[document]
        col = tbd.getcol(i)
        res = query_bow_t.dot(col)[0,0] / (col_norms.getcol(i)[0,0] * query_norm)
        if i < k:
            results.append((res, document))
        else:
            results = sorted(results)
            if res > results[0][0]:
                results[0] = (res, document)
                
    return results

In [20]:
results = find_documents_not_normalized("songwriter of Porcupine Tree", 10)
print_results(results)

Screaming Trees
Screaming Trees were an American rock band formed in Ellensburg, Washington in 1985 by vocalist Mark Lanegan, guitarist Gary Lee Conner, bass player Van Conner and drummer Mark Pickerel. Pickerel had been replaced by Barrett Martin by the time the band reached its most successful period. Although widely associated with grunge, the band's sound incorporated hard rock and psychedelic elements. During Screaming Trees' existence the band released seven studio albums, five EPs, and three compilations. Screaming Trees is known as one of the pioneers of grunge along with the Melvins, Mudhoney, U-Men, Skin Yard, Soundgarden, Green River, and Malfunkshun. Screaming Trees rose to fame as part of the grunge movement of the early 1990s, along with bands such as Alice in Chains, Pearl Jam, Nirvana, and Soundgarden and was one of the most successful underground music acts of the 1990s. The band achieved one top ten single on the Modern Rock Tracks charts. Screaming Trees were plagued

Powyższa funkcja wyszukuje k dokumentów najbardziej podobnych do podanego zapytania. Wczytywana macierz term by document ma nieznormalizowane kolumny. Przy obliczaniu korelacji kolumna macierzy i wektor bag of words wprowadzonego zapytania dzielone są przez ich normy. Funkcję przetestowano dla zapytania "songwriter of Porcupine Tree". Artykuł o Stevenie Wilsonie, którego szukano, był czwartym wynikiem wyszukiwania. Pozostałe wyniki to artykuły, w których często występują słowa "songwriter" lub "tree".

## Wyszukiwanie wykorzystując znormalizowaną macierz term by document

In [21]:
def normalize_tbd(tbd_csc):
    return normalize(tbd_csc, axis = 0)

In [22]:
tbd_csc = normalize_tbd(tbd_csc)

In [23]:
sparse.save_npz(res_dir + os.sep + 'tbd_normalized.npz', tbd_csc)

In [24]:
def find_documents_normalized(query, k):
    
    terms_dict, documents_dict = load_dicts()
    
    query_bow = query_to_bow(query, terms_dict)
    
    if query_bow == []:
        return []
    
    query_bow = normalize(query_bow, axis = 1)
    query_bow_t = query_bow.transpose()
    
    tbd = sparse.load_npz(res_dir + os.sep + 'tbd_normalized.npz')
    
    results = []
    
    for document in documents_dict.keys():
        i = documents_dict[document]
        col = tbd.getcol(i)
        res = query_bow_t.dot(col)[0,0]
        if i < k:
            results.append((res, document))
        else:
            results = sorted(results)
            if res > results[0][0]:
                results[0] = (res, document)
                
    return results

In [25]:
results = find_documents_normalized("songwriter of Porcupine Tree", 10)
print_results(results)

Screaming Trees
Screaming Trees were an American rock band formed in Ellensburg, Washington in 1985 by vocalist Mark Lanegan, guitarist Gary Lee Conner, bass player Van Conner and drummer Mark Pickerel. Pickerel had been replaced by Barrett Martin by the time the band reached its most successful period. Although widely associated with grunge, the band's sound incorporated hard rock and psychedelic elements. During Screaming Trees' existence the band released seven studio albums, five EPs, and three compilations. Screaming Trees is known as one of the pioneers of grunge along with the Melvins, Mudhoney, U-Men, Skin Yard, Soundgarden, Green River, and Malfunkshun. Screaming Trees rose to fame as part of the grunge movement of the early 1990s, along with bands such as Alice in Chains, Pearl Jam, Nirvana, and Soundgarden and was one of the most successful underground music acts of the 1990s. The band achieved one top ten single on the Modern Rock Tracks charts. Screaming Trees were plagued

W przypadku powyższej funkcji macierz term by document wczytywana z pliku jest macierzą o znormalizowanych kolumnach. Dzięki temu wyszukiwanie przebiega szybciej. Wynik zapytania dla hasła "songwriter of Procupine Tree" pokrywa się z wynikiem uzyskanym przy zastosowaniu poprzedniej funkcji.

## Wyszukiwanie z użyciem Latent Semantic Indexing

In [26]:
def apply_svd(tbd, k):
    svd = TruncatedSVD(n_components=k).fit(tbd)
    tbd = svd.transform(tbd)
    return tbd, svd

In [91]:
tbd = sparse.load_npz(res_dir + os.sep + 'tbd_normalized.npz')
tbd_trans, svd = apply_svd(tbd, 275)
sparse.save_npz(res_dir + os.sep + 'tbd_trans.npz', sparse.csc_matrix(tbd_trans))
sparse.save_npz(res_dir + os.sep + 'svd_compoments_.npz', sparse.csc_matrix(svd.components_))

Powyższa funkcja apply_svd stosuje svd i low rand approximation aby ograniczyć szum w macierzy term by document matrix.
Do zastosowania svd użyto biblioteki sklearn. Po zastosowaniu svd do plików zapisywane są macierze tbd po transformacji i svd.components_.

In [37]:
def find_documents_svd(query, k):
    
    terms_dict, documents_dict = load_dicts()
    
    query_bow = query_to_bow(query, terms_dict)
    
    if query_bow == []:
        return []
    
    query_bow = normalize(query_bow, axis = 1)
    query_bow_t = query_bow.transpose()
    query_bow_t = np.array(query_bow_t)
    tbd_trans = sparse.load_npz(res_dir + os.sep + 'tbd_trans.npz')
    svd_components = sparse.load_npz(res_dir + os.sep + 'svd_compoments_.npz')
    
    similarities = query_bow_t.dot(tbd_trans)
    similarities = similarities.dot(svd_components).getrow(0).toarray()[0]
    
    i = 0
    results = []
    for document in documents_dict.keys():
        i = documents_dict[document]
        res = similarities[i]
        if i < k:
            results.append((res, document))
        else:
            results = sorted(results)
            if res > results[0][0]:
                results[0] = (res, document)
        
    return results

In [92]:
results = find_documents_svd("songwriter of Porcupine Tree", 10)
print_results(results)

Luther Vandross
Luther Ronzoni Vandross Jr. (April 20, 1951 – July 1, 2005) was an American singer, songwriter, and record producer. Throughout his career, Vandross was an in-demand background vocalist for several different artists including Todd Rundgren, Judy Collins, Chaka Khan, Bette Midler, Diana Ross, David Bowie, Ben E. King, and Donna Summer. He later became a lead singer of the group Change, which released its gold-certified debut album, The Glow of Love, in 1980 on Warner/RFC Records. After Vandross left the group, he was signed to Epic Records as a solo artist and released his debut solo album, Never Too Much, in 1981. His hit songs include "Never Too Much", "Here and Now", "Any Love", "Power of Love/Love Power", "I Can Make It Better" and "For You to Love". Many of his songs were covers of original music by other artists such as "If This World Were Mine" (duet with Cheryl Lynn), "Since I Lost My Baby", "Superstar" and "Always and Forever". Duets such as "The Closer I Get to

In [93]:
results = find_documents_svd("black cat", 10)
print_results(results)

Black Saturday (disambiguation)
Black Saturday may refer to: Holy Saturday, the day between Good Friday and Easter Sunday Black Saturday (France), the busiest day of the year when many people go on holiday Battle of Pinkie Cleugh a 1547 battle fought between the Scottish and the English Royal armies Black Saturday (1621), a dark, stormy day in Scotland, taken as a sign of Armageddon Black Saturday (1900), the collapse of Dumbell's Bank, Isle of Man, leading to numerous bankruptcies and poverty Black Saturday (1903), the collapse of a section of balcony during a baseball game between the Boston Braves and Philadelphia Phillies, which killed 12 spectators and injured more than 200 Black Saturday (Mau Movement), a 1929 killing of 11 unarmed people by New Zealand police during a Mau demonstration in Samoa Black Saturday, a day during the 1942 Battle of Gazala between the German Afrika Korps and British armoured divisions Operation Agatha or Black Saturday (1946), British arrests of Jewish 

In [84]:
results = find_documents_normalized("songwriter of Porcupine Tree", 10)
print_results(results)

Screaming Trees
Screaming Trees were an American rock band formed in Ellensburg, Washington in 1985 by vocalist Mark Lanegan, guitarist Gary Lee Conner, bass player Van Conner and drummer Mark Pickerel. Pickerel had been replaced by Barrett Martin by the time the band reached its most successful period. Although widely associated with grunge, the band's sound incorporated hard rock and psychedelic elements. During Screaming Trees' existence the band released seven studio albums, five EPs, and three compilations. Screaming Trees is known as one of the pioneers of grunge along with the Melvins, Mudhoney, U-Men, Skin Yard, Soundgarden, Green River, and Malfunkshun. Screaming Trees rose to fame as part of the grunge movement of the early 1990s, along with bands such as Alice in Chains, Pearl Jam, Nirvana, and Soundgarden and was one of the most successful underground music acts of the 1990s. The band achieved one top ten single on the Modern Rock Tracks charts. Screaming Trees were plagued

In [79]:
results = find_documents_normalized("black cat", 10)
print_results(results)

Lewis Black
Lewis Niles Black (born August 30, 1948) is an American stand-up comedian, author, playwright, social critic and actor. His comedy routines often escalate into angry rants about history, politics, religion, or any other cultural trends. He hosted the Comedy Central series Lewis Black's Root of All Evil and makes regular appearances on The Daily Show delivering his "Back in Black" commentary segment, which he has been doing since The Daily Show was hosted by Craig Kilborn.When not on the road performing, Black resides in Manhattan, but also maintains a residence in Chapel Hill, North Carolina. He is also a spokesman for the Aruba Tourism Authority, appearing in television ads that first aired in late 2009 and 2010, as well as the voice of Anger in 2015's Pixar film, Inside Out. He was voted 51st of the 100 greatest stand-up comedians of all time by Comedy Central in 2004; he was voted 5th in Comedy Central's Stand Up Showdown in 2008 and 11th in 2010. Black has served as an 

Przetestowano różne wartości k dla SVD. Subiektywnie najlepsze wyniki były przy zastosowaniu k = 275.

Zastosowanie LSI wpłynęło znacząco na rodzaj znajdywanych artykułów. Tematycznie były bardziej powiązane z wyszukiwaną frazą. Jednak kluczowe artykuły pojawiły się dalej w wynikach, niż przy zastosowaniu jedynie macierzy znormalizowanej. Prawdopodobnie również negatywnie wpłynęła na wyszukanie konkretnych nazw własnych.

## Wpływ IDF na wyniki wyszukiwań

Sprawdzono także wpływ przekształcenia IDF na wyniki w przypadku macierzy znormalizowanej.

In [51]:
def normalize_no_idf():
    tbd_no_idf = sparse.load_npz(res_dir + os.sep + 'tbd_no_idf.npz')
    tbd_no_idf = normalize_tbd(tbd_no_idf)
    sparse.save_npz(res_dir + os.sep + 'tbd_no_idf_normalized.npz', tbd_no_idf)

    
normalize_no_idf()

In [52]:
def normalized_without_idf(query, k):
    terms_dict, documents_dict = load_dicts()
    
    query_bow = query_to_bow(query, terms_dict)
    
    if query_bow == []:
        return []
    
    query_bow = normalize(query_bow, axis = 1)
    query_bow_t = query_bow.transpose()
    
    tbd = sparse.load_npz(res_dir + os.sep + 'tbd_no_idf_normalized.npz')

    
    results = []
    
    for document in documents_dict.keys():
        i = documents_dict[document]
        col = tbd.getcol(i)
        res = query_bow_t.dot(col)[0,0]
        if i < k:
            results.append((res, document))
        else:
            results = sorted(results)
            if res > results[0][0]:
                results[0] = (res, document)
                
    return results
    

In [86]:
results = normalized_without_idf("songwriter of Porcupine Tree", 10)
print_results(results)

Screaming Trees
Screaming Trees were an American rock band formed in Ellensburg, Washington in 1985 by vocalist Mark Lanegan, guitarist Gary Lee Conner, bass player Van Conner and drummer Mark Pickerel. Pickerel had been replaced by Barrett Martin by the time the band reached its most successful period. Although widely associated with grunge, the band's sound incorporated hard rock and psychedelic elements. During Screaming Trees' existence the band released seven studio albums, five EPs, and three compilations. Screaming Trees is known as one of the pioneers of grunge along with the Melvins, Mudhoney, U-Men, Skin Yard, Soundgarden, Green River, and Malfunkshun. Screaming Trees rose to fame as part of the grunge movement of the early 1990s, along with bands such as Alice in Chains, Pearl Jam, Nirvana, and Soundgarden and was one of the most successful underground music acts of the 1990s. The band achieved one top ten single on the Modern Rock Tracks charts. Screaming Trees were plagued

In [60]:
results = normalized_without_idf("black cat", 10)
print_results(results)

Lewis Black
Lewis Niles Black (born August 30, 1948) is an American stand-up comedian, author, playwright, social critic and actor. His comedy routines often escalate into angry rants about history, politics, religion, or any other cultural trends. He hosted the Comedy Central series Lewis Black's Root of All Evil and makes regular appearances on The Daily Show delivering his "Back in Black" commentary segment, which he has been doing since The Daily Show was hosted by Craig Kilborn.When not on the road performing, Black resides in Manhattan, but also maintains a residence in Chapel Hill, North Carolina. He is also a spokesman for the Aruba Tourism Authority, appearing in television ads that first aired in late 2009 and 2010, as well as the voice of Anger in 2015's Pixar film, Inside Out. He was voted 51st of the 100 greatest stand-up comedians of all time by Comedy Central in 2004; he was voted 5th in Comedy Central's Stand Up Showdown in 2008 and 11th in 2010. Black has served as an 

Wiele wyników bez IDF pokrywa się z wynikami przy IDF. Zastosowanie IDF pozwala zminimalizować wpływ powtarzających się często słów na wynik zapytania. Duże pokrycie się wyników jest skutkiem tego, iż zebrane artykuły są dosyć zróżnicowane.

## Prosty interfejs webowy

Stworzono prosty interfejs webowy w frameworku Flask. Do jego utworzenia najważniejsze funkcje przeniesiono do odpowiednich (Search oraz TextPreprocessor). Instancja klasy Search jest tworzona przy uruchomieniu aplikacji i służy do wykonywania zapytań. Wcześniej obliczone macierze odczytuje z plików. Aplikacja webowa znajduje się w pliku search_webpage.py. Dostępne są opcje wyszukiwania: korzystając z LSI, korzystając z macierzy znormalizowanej, korzystając z macierzy nieznormalizowanej. Część wizualna zrealizowana została za pomocą Bootstrap.